<a href="https://colab.research.google.com/github/Purav0788/MuRIL3/blob/master/MuRIL_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gpustat

In [ ]:
!gpustat

39be2defab1b         Tue Nov  9 09:29:33 2021  460.32.03
[0] Tesla K80        | 70'C,   0 % |  2222 / 11441 MB |


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd
import os
os.chdir('/content/drive/My Drive/MuRIL')
!pwd

/content/drive/My Drive/MuRIL
/content/drive/My Drive/MuRIL


In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
import pandas as pd
from collections import namedtuple
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from bert import bert_tokenization
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
from keras import backend as K
import sys

TensorFlow Version: 2.6.0
Hub version:  0.12.0


In [ ]:

# Load train and val datasets
# df_full_train = pd.read_csv('gj.csv', sep = ",", engine='c', error_bad_lines=False)
df_full_train = pd.read_csv('hi.tsv', sep = "\t", engine='python', error_bad_lines=False)
df_full_train = df_full_train[(df_full_train.label != 'bioche')&(df_full_train.label !=  'com_tech')&(df_full_train.label !=  'cse')&(df_full_train.label !=  'math')&(df_full_train.label != 'mgmt')]
df_train, df_val = train_test_split(df_full_train, test_size=0.2)
# df_train, df_val = train_test_split(df_full_train, test_size=0.2)
# df_val = pd.read_csv('malayalam_dev.tsv', sep = "\t")


Skipping line 2937: '	' expected after '"'
Skipping line 3220: '	' expected after '"'
Skipping line 4170: '	' expected after '"'
Skipping line 8480: '	' expected after '"'
Skipping line 12906: '	' expected after '"'
Skipping line 17540: '	' expected after '"'
Skipping line 19442: '	' expected after '"'
Skipping line 23751: '	' expected after '"'
Skipping line 25806: '	' expected after '"'
Skipping line 31271: '	' expected after '"'
Skipping line 33531: '	' expected after '"'
Skipping line 34046: '	' expected after '"'
Skipping line 34723: '	' expected after '"'
Skipping line 35578: '	' expected after '"'
Skipping line 35816: '	' expected after '"'


In [ ]:
# Columns
df_train.columns

Index(['text', 'label'], dtype='object')

In [ ]:
# Prepare input text and one hot encoded labels for train and validation sets

# unique_labels = list(np.unique(df_train["label"]))
df_train.dropna(inplace = True)
# print(df_train["label"].unique())
unique_labels = list(np.unique(df_train.iloc[:,1]))

train_x = df_train.iloc[:,0].values
train_y = df_train.iloc[:,1].values

print(train_x)
# print(train_y)

le = preprocessing.LabelEncoder()

train_y = le.fit_transform(train_y)
train_y = tf.keras.utils.to_categorical(train_y, num_classes=len(unique_labels), dtype='float32')

val_x = df_val.iloc[:,0].values
val_y = df_val.iloc[:,1].values

val_y = le.fit_transform(val_y)
val_y = tf.keras.utils.to_categorical(val_y, num_classes=len(unique_labels), dtype='float32')


# print("number of unique labels", len(unique_labels))

['इसशलए , 1 99 0 के उत्तराधासेआर् भी आर् तक कई ऐसी कं पननयां रही हैंजर्न्होंनेव्यािसानयक रूप सेउपलबध ईंधन कोशिकाओंको बनानेकी कोशिि की है ।'
 'कार्बन एक प्रमुख चक्र है जो मानव गतिविधियों से प्रभावित होता है , और वातावरण में कार्बन डाइऑक्साइड तापमान बनाए रखने , तापमान बनाए रखने का एक कारण है और यह ऐटमौसफ़ेयर  में इसकी मात्रा द्वारा केवल 0.03 % बनता है ।'
 'तो , यह कुछ ऐसा है र्ो आप यहांदेखतेहैं ।' ...
 'इसे विराट ई खालसा कहा जाता है , यह वास्तव में एक सिख संग्रहालय था और इसे वास्तुकार मोशे सफी द्वारा डिजाइन  किया गया था ।'
 'वैज्ञानिक अनुसंधान के उत्पाद किसी विशेष समय और स्थान पर विशेष एजेंटों द्वारा बनाये जाते हैं ।'
 'यह एक और स्लाइड  है जो उत्तराखंड में लकड़ी की नक्काशी के बारे में बात करती है ।']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# Check unique labels
unique_labels

['other', 'phy']

In [ ]:
# Function to create input_ids
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

# Function to create attention masks
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

# Function to create segment ids
def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

# Function to create input_ids, attention_masks, segment_ids for sample
def create_single_input(sentence,MAX_LEN, MAX_SEQ_LEN):
  
  stokens = tokenizer.tokenize(sentence)
  
  stokens = stokens[:MAX_LEN]
  
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)

  return ids,masks,segments

def create_input_array(sentences, MAX_SEQ_LEN):

  input_ids, input_masks, input_segments = [], [], []

  for sentence in tqdm(sentences,position=0, leave=True):
  
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2, MAX_SEQ_LEN)

    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [ ]:
# MuRIL model layer
muril_layer = hub.KerasLayer("https://tfhub.dev/google/MuRIL/1", trainable=False)

# Create tokenizer
vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
# Create input_ids, attention_masks, segment_ids for training and validation sets with max_seq_len as 128
max_seq_len = 128
train_x = create_input_array(train_x, max_seq_len)
val_x = create_input_array(val_x, max_seq_len)

100%|██████████| 2526/2526 [00:01<00:00, 2271.50it/s]


In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
  def on_train_batch_end(self, batch, logs = None):
    print(batch)

In [ ]:
my_callbacks = [
    # tf.keras.callbacks.EarlyStopping(patience=2),
    # tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [ ]:
# Define model function - compile and fit
def model_fit(train_x, train_y, val_x, val_y, max_seq_length, num_epochs, muril_layer):

  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
  segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
  
  outputs = muril_layer(dict(input_word_ids = input_word_ids, input_mask = input_mask, input_type_ids = segment_ids))

  # my_outputs = K.print_tensor(outputs, message ="muril Layer encodings")
  print(outputs["sequence_output"].shape)
  print(outputs["pooled_output"].shape)
  # model.summary()
  x = tf.keras.layers.Dropout(0.2)(outputs["pooled_output"]) # take pooled output layer
  # print(x)
  final_output = tf.keras.layers.Dense(len(unique_labels), activation="softmax", name="dense_output")(x)
  my_final_output = K.print_tensor(final_output)
  # tf.print(final_output, output_stream = sys.stderr)
  model = tf.keras.models.Model(
      inputs=[input_word_ids, input_mask, segment_ids], outputs=my_final_output)
  opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=True, name="SGD")
  opt = tfa.optimizers.Lookahead(opt)
  model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['f1_score'])
  K.set_value(model.optimizer.learning_rate, 0.001)
  model.summary()
  model.fit(train_x, train_y, epochs = num_epochs, batch_size = 32, validation_data = (val_x, val_y), shuffle = True, callbacks = my_callbacks)
  return model

In [ ]:
# Define model function - compile and fit
def model_fit(train_x, train_y, val_x, val_y, max_seq_length, num_epochs, muril_layer):

  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
  segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
  
  outputs = muril_layer(dict(input_word_ids = input_word_ids, input_mask = input_mask, input_type_ids = segment_ids))

  x = tf.keras.layers.Dropout(0.2)(outputs["pooled_output"]) # take pooled output layer
  final_output = tf.keras.layers.Dense(len(unique_labels), activation="softmax", name="dense_output")(x)

  model = tf.keras.models.Model(
      inputs=[input_word_ids, input_mask, segment_ids], outputs=final_output)

  model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
  
  # Define the Keras TensorBoard callback.
  logdir="logs/fit/metrics"
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

  history = model.fit(train_x, train_y, epochs = num_epochs, batch_size = 32, validation_data = (val_x, val_y), shuffle = True, callbacks=[tensorboard_callback])

  return history

In [ ]:
# Set number of epochs
num_epochs = 1

# Get the model object
model = model_fit(train_x, train_y, val_x, val_y, max_seq_len, num_epochs, muril_layer)



316/316 [==============================] - 268s 813ms/step - loss: 0.6902 - accuracy: 0.5394 - val_loss: 0.6889 - val_accuracy: 0.5400


In [ ]:
# Make predictions
preds = model.predict(val_x)
print(preds)

AttributeError: ignored